This project explores the following aspects of San Francisco: crimes, schools, housing prices and food truck.

In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np

In [2]:
con = sql.connect('sf_data.sqlite')

In [3]:
# display all the tables in the database
pd.read_sql("SELECT name FROM sqlite_master WHERE type = 'table'", con)

,name
0,crime
1,noise
2,parking
3,schools
4,parks
5,zillow
6,mobile_food_permits
7,mobile_food_locations
8,mobile_food_schedule
9,crime_count


1. Find out the most severe crime category in each district

In [4]:
pd.read_sql("SELECT PdDistrict, Crime_Category, MAX(Num_Crime_Category) num_crime FROM "
            "(SELECT PdDistrict, Category AS Crime_Category, COUNT(Category) AS Num_Crime_Category FROM crime "
            "WHERE PdDistrict != 'None' GROUP BY PdDistrict, Category "
            "ORDER BY COUNT(Category) DESC) GROUP BY PdDistrict ORDER BY num_crime DESC", con)

,PdDistrict,Crime_Category,num_crime
0,SOUTHERN,LARCENY/THEFT,60068
1,NORTHERN,LARCENY/THEFT,39247
2,CENTRAL,LARCENY/THEFT,36299
3,MISSION,LARCENY/THEFT,23706
4,BAYVIEW,OTHER OFFENSES,18791
5,TARAVAL,LARCENY/THEFT,15777
6,RICHMOND,LARCENY/THEFT,15189
7,INGLESIDE,OTHER OFFENSES,15125
8,PARK,LARCENY/THEFT,13448
9,TENDERLOIN,DRUG/NARCOTIC,12257


2. Find out the most dangerous day of the week in each district and the total number of crimes occur in that day 

In [5]:
pd.read_sql("SELECT PdDistrict, DayOfWeek, MAX(num_crime) AS total_crime "
            "FROM(SELECT PdDistrict, DayOfWeek, COUNT(Descript) AS num_crime FROM crime WHERE PdDistrict != 'None' "
            "GROUP BY PdDistrict, DayOfWeek ORDER BY COUNT(Descript) DESC) GROUP BY PdDistrict ORDER BY total_crime DESC", con)

,PdDistrict,DayOfWeek,total_crime
0,SOUTHERN,Friday,30952
1,MISSION,Friday,20568
2,NORTHERN,Friday,19353
3,CENTRAL,Saturday,18211
4,BAYVIEW,Friday,15816
5,INGLESIDE,Friday,13485
6,TENDERLOIN,Wednesday,13079
7,TARAVAL,Friday,11713
8,PARK,Friday,9338
9,RICHMOND,Friday,8728


3. Find out the neighborhood that schools locate

In [6]:
school_neighborhood = pd.read_sql("SELECT DISTINCT(schools.Name), SUBSTR(schools.Address, -5) AS zip_code, noise.Neighborhood FROM schools "
            "INNER JOIN noise ON ROUND(schools.Lon, 3) = ROUND(noise.Lon, 3) "
            "AND ROUND(schools.Lat, 3) = ROUND(noise.Lat, 3)", con)

In [7]:
# Only shows top 5 due to space
school_neighborhood.head()

,Name,zip_code,Neighborhood
0,Alvarado Elementary School,94114,Noe Valley
1,"Carmichael, Bessie Carmichael (6-8 Campus)",94107,South of Market
2,"Carmichael, Bessie Carmichael K-5 Campus / Ear...",94103,South of Market
3,"Carver, Dr. George Washington Carver Elementar...",94124,Bayview
4,"Chin, John Yehall Chin Elementary School",94133,North Beach


In [8]:
school_neighborhood.to_sql("school_neighborhood", con, if_exists = 'replace', index = False)

4. Find out the TOP 10 schools that locate in the area that has the high average median sold price. 

In [9]:
pd.read_sql("SELECT Name AS school_name, Avg_Median_Price_PerSqft FROM "
            "(SELECT RegionName, AVG(MedianSoldPricePerSqft_AllHomes * ZriPerSqft_AllHomes) AS Avg_Median_Price_PerSqft "
            "FROM zillow GROUP BY RegionName) AS Z INNER JOIN school_neighborhood AS SN "
            "ON Z.RegionName = SN.zip_code ORDER BY Avg_Median_Price_PerSqft DESC LIMIT 10", con)

,school_name,Avg_Median_Price_PerSqft
0,Marin Day School - Spear St,4331.122481
1,Marina Middle School,3614.991055
2,Sherman Elementary School,3614.991055
3,Amici World School,3614.991055
4,Hergl School,3614.991055
5,Cross Cultural Family Center - Mary Lane Infan...,3614.991055
6,Marina Childrens Center,3614.991055
7,Larkin Street Youth Services Academy,3230.957820
8,De Marillac Academy,3230.957820
9,Proof School,3230.957820


5. What is the best times and places to find food trucks?

In [10]:
pd.read_sql("SELECT tmp_t.locationid, Address, MAX(num_food) AS num_food_truck, DayOfWeek FROM "
            "(SELECT locationid, Address, COUNT(locationid) as num_food, "
            "ROUND(Latitude, 2) AS Lat, ROUND(Longitude, 2) AS Lon "
            "FROM mobile_food_locations GROUP BY Lat, Lon) AS tmp_t "
            "INNER JOIN mobile_food_schedule AS mfs ON tmp_t.locationid = mfs.locationid "
            "GROUP BY tmp_t.locationid, DayOfWeek HAVING num_food_truck > 30 ORDER BY num_food_truck DESC", con)

,locationid,Address,num_food_truck,DayOfWeek
0,850007,225 BATTERY ST,111,Fr
1,850007,225 BATTERY ST,111,Mo
2,850007,225 BATTERY ST,111,We
3,936625,170 OTIS ST,58,Fr
4,936625,170 OTIS ST,58,Mo
5,936625,170 OTIS ST,58,Th
6,936625,170 OTIS ST,58,Tu
7,936625,170 OTIS ST,58,We
8,936872,1301 CESAR CHAVEZ ST,45,Fr
9,936872,1301 CESAR CHAVEZ ST,45,Mo


6. Which mobile food vendor(s) sells at the most locations?

In [11]:
# Join the mobile_food_tables
vendors = pd.read_sql("SELECT FacilityType, Applicant, Address FROM mobile_food_schedule "
            "INNER JOIN mobile_food_locations ON mobile_food_schedule.locationid = mobile_food_locations.locationid "
            "INNER JOIN mobile_food_permits ON mobile_food_schedule.permit = mobile_food_permits.permit", con)

In [12]:
# Create a vendors table in the database
vendors.to_sql("vendors", con, if_exists = 'replace', index = False)

In [13]:
# Get the food vendor that sells at the most location
vendors_loc = pd.read_sql("SELECT FacilityType, Applicant, Address, COUNT(DISTINCT (Address)) AS Unique_Address_Count FROM vendors "
                          "GROUP BY Applicant ORDER BY Unique_Address_Count DESC LIMIT 1", con)

In [14]:
vendors_loc

,FacilityType,Applicant,Address,Unique_Address_Count
0,Truck,May Catering,1059 EVANS AVE,58


In [15]:
con.close()